In [1]:
import numpy as np
import math



In [2]:
q_home = [180,10,280,135,180]

LEN_A = 115.432 # Height offset of base
LEN_B = 41.0513 # Translation (x) offset of base rotation joint (1st joint)
LEN_C = 168.8579 # Length of first arm link (joint 2 to 3)
LEN_D = 127.9234 # Length of secon arm link (joint 3 to 4)

LEN_E = 108.5357 # Length of gripper (from joint4 to grasping poit)
LEN_F = 7.7076 # Z Offset of gripper

LEN_G = 90.0 # Offset (degree) of last rotaitonal joint (joint 4) of gripper
LEN_H = -13.75 # shift along y of 1st rotational axis (base, joint 1) of arm. Arm is mounted "off-center".

initPosX = LEN_B + LEN_D + LEN_E
initPosY = LEN_H
initPosZ = LEN_A + LEN_C - LEN_F
initPosT = 90

rateTZ = 2


# joint angles to servo positions!!
# void ctrlMove(){
#   STPos[0] = (int)(angleGenOut(STDirection[0]*InfoBuffer[angle_1]) + 0.5);
#   STPos[1] = (int)(angleGenOut(STDirection[1]*InfoBuffer[angle_2]+15)*3 + 0.5);
#   STPos[2] = (int)(angleGenOut(STDirection[2]*InfoBuffer[angle_3]+180)  + 0.5);
#   STPos[3] = (int)(angleGenOut(STDirection[3]*InfoBuffer[angle_4]+180)  + 0.5);
#   st.SyncWritePosEx(STID, 4, STPos, STSpd, STAc);
# }


# Servo pos to joint angles:
# void getAngTorInfo(){
#   if(st.FeedBack(1)!=-1){
#     jsonStInfoSend["A1"] = angleGenOutReverse(st.ReadPos(-1));
#     jsonStInfoSend["T1"] = st.ReadLoad(-1);
#   }
#   if(st.FeedBack(2)!=-1){
#     jsonStInfoSend["A2"] = angleGenOutReverse(st.ReadPos(-1))/3-15;
#     jsonStInfoSend["T2"] = st.ReadLoad(-1);
#   }
#   if(st.FeedBack(3)!=-1){
#     jsonStInfoSend["A3"] = angleGenOutReverse(st.ReadPos(-1))-180;
#     jsonStInfoSend["T3"] = st.ReadLoad(-1);
#   }
#   if(st.FeedBack(4)!=-1){
#     jsonStInfoSend["A4"] = STDirection[3]*(angleGenOutReverse(st.ReadPos(-1))-180);
#     jsonStInfoSend["T4"] = st.ReadLoad(-1);
#   }
#   if(st.FeedBack(5)!=-1){
#     jsonStInfoSend["A5"] = angleGenOutReverse(st.ReadPos(-1));
#     jsonStInfoSend["T5"] = st.ReadLoad(-1);
#   }


In [3]:
# Inverse kinematic routines as provided by waveshare:

def simpleLinkageIK(LA, LB, aIn, bIn):
    if (bIn == 0):
        psi   = math.acos((LA * LA + aIn * aIn - LB * LB)/(2 * LA * aIn)) * 180 / math.pi
        alpha = 90 - psi
        omega = math.acos((aIn * aIn + LB * LB - LA * LA)/(2 * aIn * LB)) * 180 / math.pi
        beta  = psi + omega
    else:
        L2C = aIn * aIn + bIn * bIn
        LC  = math.sqrt(L2C)
        lamb = math.atan(bIn/aIn) * 180 / math.pi;
        psi    = math.acos((LA * LA + L2C - LB * LB)/(2 * LA * LC)) * 180 / math.pi
        alpha = 90 - lamb - psi
        omega = math.acos((LB * LB + L2C - LA * LA)/(2 * LC * LB)) * 180 / math.pi
        beta  = psi + omega

    delta = 90 - alpha - beta

    if (math.isnan(alpha) or math.isnan(beta) or math.isnan(delta)):
        raise Exception("NAN")
       
    angle_2   = alpha;
    angle_3   = beta;
    angle_IKE = delta;
    return angle_2, angle_3, angle_IKE



def EoAT_IK(angleInput):
    print("angleInput",angleInput)
    if (angleInput == 90):
        betaGenOut = angleInput - LEN_G
        betaRad    = betaGenOut*math.pi/180
        angleRad   = angleInput*math.pi/180
        aGenOut = LEN_E;
        bGenOut = LEN_F;
    elif (angleInput < 90):
        betaGenOut = 90 - angleInput
        betaRad    = betaGenOut*math.pi/180
        angleRad   = angleInput*math.pi/180
        aGenOut = math.cos(angleRad)*LEN_F + math.cos(betaRad)*LEN_E
        bGenOut = math.sin(angleRad)*LEN_F - math.sin(betaRad)*LEN_E
        betaGenOut = -betaGenOut
    elif (angleInput > 90):
        betaGenOut = LEN_G - (180 -angleInput)
        betaRad    = betaGenOut*math.pi/180
        angleRad   = angleInput*math.pi/180
        aGenOut = - math.cos(math.pi-angleRad)*LEN_F + math.cos(betaRad)*LEN_E
        bGenOut = math.sin(math.pi-angleRad)*LEN_F + math.sin(betaRad)*LEN_E
    print("betaGenOut",betaGenOut)
  
    if (math.isnan(betaGenOut)):
        raise Exception("NAN")
        
        
    angle_EoAT = betaGenOut
    len_a = aGenOut;
    len_b = bGenOut;
    return angle_EoAT, len_a, len_b



def wigglePlaneIK(LA, aIn, bIn):
    bIn = -bIn
    if (bIn > 0):
        L2C = aIn * aIn + bIn * bIn
        LC = math.sqrt(L2C)
        lamb = math.atan(aIn/bIn) * 180 / math.pi
        psi = math.acos(LA/LC) * 180 / math.pi
        LB = math.sqrt(L2C - LA * LA)
        alpha = psi + lamb - 90
    elif (bIn == 0):
        alpha = 90+ math.asin(LA/aIn) * 180 / math.pi
        L2C = aIn * aIn + bIn * bIn
        LB = math.sqrt(L2C)
    elif (bIn < 0):
        bIn = -bIn;
        L2C = aIn * aIn + bIn * bIn
        LC = math.sqrt(L2C)
        lamb = math.atan(aIn/bIn) * 180 / math.pi
        psi = math.acos(LA/LC) * 180 / math.pi
        LB = math.sqrt(L2C - LA * LA)
        alpha = 90 - lamb + psi
        
    if (math.isnan(alpha)):
        raise Exception("NAN")
        
    angle_1 = alpha + 90
    len_totalXY = LB - LEN_B
    return angle_1, len_totalXY

def ikin(InputX, InputY, InputZ, InputTheta):
    InputTheta = InputTheta/rateTZ
    angle_1, len_totalXY = wigglePlaneIK(LEN_H, InputX, InputY)
    print("angle_1, len_totalXY",angle_1, len_totalXY)
    angle_EoAT, len_a, len_b = EoAT_IK(InputTheta)
    print("angle_EoAT, len_a, len_b",angle_EoAT, len_a, len_b)
    print("len_totalXY-len_a",len_totalXY-len_a)
    print("InputZ - LEN_A + len_b",InputZ - LEN_A + len_b)
    angle_2, angle_3, angle_IKE = simpleLinkageIK(LEN_C, LEN_D, (len_totalXY-len_a), (InputZ - LEN_A + len_b))
    angle_4 = angle_IKE + angle_EoAT
    
    return angle_1, angle_2, angle_3, angle_4


In [4]:
print("ikin result is:", ikin(20,20,20, 172))

angle_1, len_totalXY 254.0869211118614 -16.334150083393514
angleInput 86.0
betaGenOut -4.0
angle_EoAT, len_a, len_b -4.0 108.80896748681542 0.11775696643235989
len_totalXY-len_a -125.14311757020893
InputZ - LEN_A + len_b -95.31424303356764
ikin result is: (254.0869211118614, 6.6911120708836975, 117.77460698146626, -38.465719052349954)


In [5]:
# Reconstructed forward kinematic calculation:

def rotate(point, angle, origin=(0,0)):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in degree.
    """
    ox, oy = origin
    px, py = point
    
    angle = angle * math.pi/180

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def translate(point, offset):
    """
    Translate point by offset.

    """
    return point[0]+offset[0], point[1]+offset[1]


p=[0,0]

rotate((1,0), 90, p)
translate((1,0), p)


def fkin(angle_1, angle_2, angle_3, angle_4):
    p=(0,0)
    p = translate(p, (LEN_E,-LEN_F))

    p = rotate(p, -angle_4)

    p = translate(p, (LEN_D,0))

    p = rotate(p, -angle_3)

    p = translate(p, (LEN_C,0))

    p = rotate(p, 90-angle_2)
 
    p = translate(p, (LEN_B,LEN_A))

    
    print(angle_1)
    p_plane = rotate((p[0],LEN_H), angle_1-180)
    
    return (p_plane[0], p_plane[1], p[1])

    





In [6]:
ang = 170
qs = ikin(LEN_B+LEN_C+LEN_D+LEN_E,LEN_H,LEN_A-LEN_F, ang)
print("ang:",ang/2-90)
print(qs)
fkin(*qs)

angle_1, len_totalXY 180.00000000000003 405.317
angleInput 85.0
betaGenOut -5.0
angle_EoAT, len_a, len_b -5.0 108.79445049647812 -1.781239293125065
len_totalXY-len_a 296.5225495035219
InputZ - LEN_A + len_b -9.488839293125064
ang: -5.0
(180.00000000000003, 90.49394426738046, 3.106398789519801, -8.600343056900257)
180.00000000000003


(446.3683, -13.749999999999778, 107.72440000000044)

In [10]:

#qs_p = [(LEN_B+LEN_C+LEN_D+LEN_E),LEN_H,LEN_A-LEN_F, 180]
#qs_p = [(LEN_B+LEN_C+LEN_D+LEN_E)/2,LEN_H,LEN_A-LEN_F, 180]
qs_p = [(LEN_B+LEN_C+LEN_D+LEN_E)/2,LEN_H,LEN_A-LEN_F+50, 150]
#qs_p = [(LEN_B+LEN_C+LEN_D+LEN_E)/2,LEN_H,LEN_A-LEN_F, 170]

qs = ikin(*qs_p)
qs=[qs[0],qs[1],qs[2],qs[3]]
print("angles:",qs)

print("calc: ", fkin(*qs))
print("orig: ", *qs_p[:-1] )


angle_1, len_totalXY 180.0 182.13285000000002
angleInput 75.0
betaGenOut -15.0
angle_EoAT, len_a, len_b -15.0 106.83230937639462 -20.64613633482804
len_totalXY-len_a 75.3005406236054
InputZ - LEN_A + len_b 21.64626366517196
angles: [180.0, 27.68150627356794, 153.72655942136532, -106.40806569493327]
180.0
calc:  (223.18415000000005, -13.75, 157.7244)
orig:  223.18415000000002 -13.75 157.7244


In [104]:
LEN_B+LEN_C+LEN_D+LEN_F

345.5402

In [159]:
LEN_A

115.432

In [100]:
LEN_B+LEN_C+LEN_D+LEN_E

446.36830000000003

In [129]:
LEN_B+LEN_C+LEN_D

337.8326

In [136]:
ikin(LEN_B+LEN_C+LEN_D+LEN_E,LEN_H,LEN_A+LEN_F, 180)

angle_1, len_totalXY 180.00000000000003 405.317
angle_EoAT, len_a, len_b 0.0 108.5357 7.7076
len_totalXY-len_a 296.7813
InputZ - LEN_A + len_b 15.415199999999999


ValueError: math domain error

In [123]:
LEN_B+LEN_C+LEN_D

337.8326